In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import time

import torch as T
import torch.nn as nn
import torch.nn.functional as F
from model import Model

from data_util import config, data
from data_util.batcher import Batcher
from data_util.data import Vocab
from train_util import *
from beam_search import *
from rouge import Rouge
import argparse;

In [4]:
def get_cuda(tensor):
    if T.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

In [18]:
class Evaluate(object):
    def __init__(self, data_path, opt, batch_size = config.batch_size):
        self.vocab = Vocab(config.vocab_path, config.vocab_size)
        self.batcher = Batcher(data_path, self.vocab, mode='eval',
                               batch_size=batch_size, single_pass=True)
        self.opt = opt
        time.sleep(5)

    def setup_valid(self):
        self.model = Model()
        self.model = get_cuda(self.model)
        checkpoint = T.load(os.path.join(config.save_model_path, self.opt.load_model))
        self.model.load_state_dict(checkpoint["model_dict"])


    def print_original_predicted(self, decoded_sents, ref_sents, article_sents, loadfile):
        filename = "test_"+loadfile.split(".")[0]+".txt"
    
        with open(os.path.join("data",filename), "w") as f:
            for i in range(len(decoded_sents)):
                f.write("article: "+article_sents[i] + "\n")
                f.write("ref: " + ref_sents[i] + "\n")
                f.write("dec: " + decoded_sents[i] + "\n\n")

    def evaluate_batch(self, print_sents = False):

        self.setup_valid()
        batch = self.batcher.next_batch()
        start_id = self.vocab.word2id(data.START_DECODING)
        end_id = self.vocab.word2id(data.STOP_DECODING)
        unk_id = self.vocab.word2id(data.UNKNOWN_TOKEN)
        decoded_sents = []
        ref_sents = []
        article_sents = []
        rouge = Rouge()
        count = 0
        while batch is not None and count < 20:
            enc_batch, enc_lens, enc_padding_mask, enc_batch_extend_vocab, extra_zeros, ct_e = get_enc_data(batch)

            with T.autograd.no_grad():
                enc_batch = self.model.embeds(enc_batch)
                enc_out, enc_hidden = self.model.encoder(enc_batch, enc_lens)

            #-----------------------Summarization----------------------------------------------------
            with T.autograd.no_grad():
                pred_ids = beam_search(enc_hidden, enc_out, enc_padding_mask, ct_e, extra_zeros, enc_batch_extend_vocab, self.model, start_id, end_id, unk_id)

            for i in range(len(pred_ids)):
                decoded_words = data.outputids2words(pred_ids[i], self.vocab, batch.art_oovs[i])
                if len(decoded_words) < 2:
                    decoded_words = "xxx"
                else:
                    decoded_words = " ".join(decoded_words)
                decoded_sents.append(decoded_words)
                abstract = batch.original_abstracts[i]
                article = batch.original_articles[i]
                ref_sents.append(abstract)
                article_sents.append(article)

            batch = self.batcher.next_batch()
            count += 1
            print(batch.original_abstracts)

        load_file = self.opt.load_model

        if print_sents:
            self.print_original_predicted(decoded_sents, ref_sents, article_sents, load_file)

        scores = rouge.get_scores(decoded_sents, ref_sents, avg = True)
        if self.opt.task == "test":
            print(load_file, "scores:", scores)
        else:
            rouge_l = scores["rouge-l"]["f"]
            print(load_file, "rouge_l:", "%.4f" % rouge_l)

In [19]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [21]:
task = "validate"
start_from = "0000001.tar" #name of first model -- change this
load_model = None

opt = Namespace(task = task, start_from = start_from, load_model = load_model)

if (opt.task == "validate"):
    saved_models = os.listdir(config.save_model_path)
    saved_models.sort()
    file_idx = saved_models.index(start_from)
    saved_models = saved_models[file_idx:]
    for f in saved_models:
        opt.load_model = f
        eval_processor = Evaluate(config.valid_data_path, opt)
        eval_processor.evaluate_batch(print_sents = True) 
else:   #test
    eval_processor = Evaluate(config.test_data_path, opt)
    eval_processor.evaluate_batch()



Exception in thread Thread-16:
Traceback (most recent call last):
  File "C:\Users\georg\w210\capstone\LSTM_Summarizer\data_util\batcher.py", line 272, in text_generator
    e = next(example_generator) # e is a tf.Example
StopIteration

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\georg\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\georg\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\georg\w210\capstone\LSTM_Summarizer\data_util\batcher.py", line 206, in fill_example_queue
    (article, abstract) = next(input_gen) # read the next example from file. article and abstract are both strings.
RuntimeError: generator raised StopIteration



data/saved_models
['egypt votes amid strife and charges of fraud david d. kirkpatrick contributed reporting from alexandria egypt', 'distinct signature found in #### anthrax attack', 'lawyer says dna tests show #rd person at sheppard scene', 'new york governor to change tune on freedom tower', 'fidelity says tax breaks would prevent moves', 'woods driving toward a masters title', '<unk> <unk> ## yachtsman lost at sea', 'search for <unk> the penguin takes a dark turn', 'new york better equipped for bio-terror attack before gop convention', 'argentina # croatia # in davis cup world group play', 'virginia fields extends her political reach', 'angela s ashes author mccourt dies in nyc at ##', 'indonesian retail trade predicted to go sluggish in ####', 'a look at alstom maker of trains and cruise ships with <unk> <unk>', "on the eve of powell 's visit arab leaders under pressure from", 'ford withheld info about defective mechanism lawsuit asserts', 'while kidd visits spurs parker ponders hi

["children 's safety often neglected in cars", 'sci-fi jeremiah suffers from lack of energy', 'finnish bank unitas reports loss for first quarter', '<unk> tour of dance courtesy of mark morris', "sir edmund hillary 's son to attempt everest summit again", 'swedish museum body rejects offer to buy back stolen <unk>', 'german public defict ratio stands #.# pct of gdp in ####', 'iraqi cash spent on rebuilding', 'amos oz and other israeli authors pick olives with besieged palestinians', '#### int l cycle show to open at nangang exhibition hall march ##', 'subway fire attacker known official says', 'security tightened across indonesia amid fears of more terrorist attacks', 'jiang zemin to attend summit meeting in kyrgyz', 'un investigator urges philippine military to admit facts of extrajudicial killings', 'night club owner tries to block crowd size testimony in fire trial', 'zimbabwean lawyers prepare lawsuits against mass demolitions evictions', 'bush signs identity theft bill', 'stolen r

KeyboardInterrupt: 